In [ ]:
# Regrid RR w data to 0.1 deg grid

    # Regrid lat lon to 0.1 deg over 20 to 55N, to 150 to 50W
lat2d = np.linspace(20,55,351)
lon2d = np.linspace(210,310,1001)

X, Y = np.meshgrid(lon2d,lat2d)

# Need the model lat/lon/lev values for regridding
mdllat = ds_rr['lat']
mdllon = ds_rr['lon']
mdlev = ds_rr['lev']


# get time 
# rr_time = ds_rr['time']

In [ ]:
# loop over number of levels and interpolate 1D array to 3D array with dimensions lat, lon, lev
# WARNING - executing this cell will reinterpolate each of 110 levels and take ~ 2 mins to execute

levdata = []
tic = time.time()
for i in range(len(mdlev)):
    
    inter1 = time.time()
    
    levdata.append(griddata((mdllon,mdllat), ds_rr.isel(time=0,lev=i)['OMEGA'], (X, Y), method='linear'))
    
    inter2 = time.time()
    
    print("interpolating level ", i, " took ", inter2 - inter1, " seconds")


toc = time.time()

# loop over time and add data to list to now have time, lev, lat, lon
print("Whole process took ", toc - tic, " seconds")

In [ ]:
# WARNING - executing this line more than once will raise Errno 13 Permission denied error
# as file is already created. I'm sure this can be written over somehow.

ds_rr_regrid1_omega = Dataset('./Data/CONUS_0.1x0.1_regrid1_omega.nc', 'w', format='NETCDF4')

# Create dimensions of the netcdf: 
# lat, lon, lev in accordance with "regrid1" 
    # lat lon grid with 0.1 deg spacing in the range 20 - 55 N, 210 - 310 E
    # 0.1 in both lat,lon 
    # Level structure as defined in CONUS RR run
    
lat = ds_rr_regrid1_omega.createDimension('lat', np.size(lat2d))
lon = ds_rr_regrid1_omega.createDimension('lon', np.size(lon2d))
lev = ds_rr_regrid1_omega.createDimension('lev', np.size(mdlev))
# time = 

# Create variables, this includes dimensions and fields
lats = ds_rr_regrid1_omega.createVariable('lat', 'f8', ('lat'))
lons = ds_rr_regrid1_omega.createVariable('lon', 'f8', ('lon'))
levs = ds_rr_regrid1_omega.createVariable('lev', 'f8', ('lev'))
# times = ...

omega = ds_rr_regrid1_omega.createVariable('omega', 'f8', ('lat','lon','lev'))

omega.units = "Pa/s"
omega.longname = "Vertical velocity in pressure coordinates"

# ds_rr_regrid1_omega.attrs = ds_rr.attrs

In [ ]:
temp = np.transpose(levdata, (1,2,0))
# Assign the values from the numpy arrays we have calculated above
lats[:] = lat2d
lons[:] = lon2d
levs[:] = mdlev
omega[:,:,:] = temp